# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

# Import original data and preprocessed data

In [2]:
INPUT_DIR = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files'
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')

### Note: This Step is for the extraction of California State and Hobbies category

In [3]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [4]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Hobbies\Demand Pattern Categorization'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [5]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
10,HOBBIES_1_011_CA_1_validation,HOBBIES_1_011,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
17,HOBBIES_1_018_CA_1_validation,HOBBIES_1_018,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,HOBBIES_1_026_CA_1_validation,HOBBIES_1_026,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34,HOBBIES_1_035_CA_1_validation,HOBBIES_1_035,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,HOBBIES_1_039_CA_1_validation,HOBBIES_1_039,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9702,HOBBIES_2_140_CA_4_validation,HOBBIES_2_140,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9705,HOBBIES_2_143_CA_4_validation,HOBBIES_2_143,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9706,HOBBIES_2_144_CA_4_validation,HOBBIES_2_144,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# User-defined functions to calculate Metrics and Algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [8]:
sales_pattern = sales_intermittent.copy()

In [9]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)

dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [10]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [11]:
list_params_alpha = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]
list_params_beta = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [12]:
len(list_pattern_py)

1301

In [13]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

dataframe_check = dataframe_weekly[sample_list].iloc[-10:-2, :].T
dataframe_check['Check'] = np.where((dataframe_check==0).all(axis=1), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [14]:
len(sample_list_filtered)

121

# Hyperparameter tuning for Croston TSB

In [11]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [16]:
start = timer()

df_result_all_params = pd.DataFrame()

for product in sample_list_filtered:
    
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    
    for i in list_params_alpha:
        for j in list_params_beta:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in train_second.index.tolist():
                yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                           'Actual Data': train_second,
                                           'Forecast': predictions,
                                           'Alpha': [i for count_i in range(len(train_second))],
                                           'Beta': [j for count_j in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)    
            
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_1_168_CA_4
Progressing: 0.0 %
Currently Running: FOODS_3_182_CA_4
Progressing: 0.83 %
Currently Running: FOODS_2_025_CA_4
Progressing: 1.65 %
Currently Running: FOODS_1_156_CA_1
Progressing: 2.48 %
Currently Running: FOODS_3_778_CA_4
Progressing: 3.31 %
Currently Running: FOODS_2_305_CA_4
Progressing: 4.13 %
Currently Running: FOODS_1_137_CA_1
Progressing: 4.96 %
Currently Running: FOODS_3_528_CA_2
Progressing: 5.79 %
Currently Running: FOODS_3_374_CA_1
Progressing: 6.61 %
Currently Running: FOODS_3_315_CA_3
Progressing: 7.44 %
Currently Running: FOODS_1_191_CA_2
Progressing: 8.26 %
Currently Running: FOODS_2_001_CA_3
Progressing: 9.09 %
Currently Running: FOODS_2_042_CA_4
Progressing: 9.92 %
Currently Running: FOODS_3_465_CA_1
Progressing: 10.74 %
Currently Running: FOODS_2_170_CA_4
Progressing: 11.57 %
Currently Running: FOODS_3_357_CA_4
Progressing: 12.4 %
Currently Running: FOODS_1_118_CA_3
Progressing: 13.22 %
Currently Running: FOODS_3_504_CA_1
Progressin

In [17]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,FOODS_1_168_CA_4,1.0,1.856724,0.05,0.05
1,FOODS_1_168_CA_4,2.0,1.821785,0.05,0.05
2,FOODS_1_168_CA_4,3.0,1.831386,0.05,0.05
3,FOODS_1_168_CA_4,1.0,1.884326,0.05,0.05
4,FOODS_1_168_CA_4,1.0,1.846771,0.05,0.05
...,...,...,...,...,...
313627,FOODS_3_684_CA_1,0.0,3.856269,0.90,0.90
313628,FOODS_3_684_CA_1,5.0,0.385627,0.90,0.90
313629,FOODS_3_684_CA_1,10.0,4.445948,0.90,0.90
313630,FOODS_3_684_CA_1,9.0,9.403168,0.90,0.90


# Summay all metrics

In [12]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [19]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.0067448799999994 minutes


In [20]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_030_CA_4,0.05,0.05,4.377074,666.9827,160.2415,180.4032,2.501185,2.573704
1,FOODS_1_030_CA_4,0.05,0.10,3.269276,498.1754,151.0478,111.6875,1.868158,1.987769
2,FOODS_1_030_CA_4,0.05,0.15,2.519510,383.9253,142.1232,64.9285,1.439720,1.606810
3,FOODS_1_030_CA_4,0.05,0.20,2.030045,309.3402,135.7418,36.6697,1.160026,1.357106
4,FOODS_1_030_CA_4,0.05,0.25,1.785652,272.0993,135.8967,29.3265,1.020372,1.211201
...,...,...,...,...,...,...,...,...,...
39199,FOODS_3_824_CA_1,0.90,0.70,0.931928,106.5060,179.7013,73.1753,1.863856,4.614195
39200,FOODS_3_824_CA_1,0.90,0.75,0.931712,106.4814,180.2083,76.3210,1.863424,4.615406
39201,FOODS_3_824_CA_1,0.90,0.80,0.931939,106.5073,180.7150,79.6360,1.863878,4.616661
39202,FOODS_3_824_CA_1,0.90,0.85,0.932634,106.5868,181.2270,83.1700,1.865269,4.618048


# Check what products has unexpected metrics result

In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [23]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
22680,FOODS_3_206_CA_2,0.05,0.05,1.111654,110.4749,NaN,96.0573,3.176155,4.657215
22681,FOODS_3_206_CA_2,0.05,0.10,1.182910,117.5563,NaN,92.3867,3.379744,4.566417
22682,FOODS_3_206_CA_2,0.05,0.15,1.227295,121.9672,NaN,88.8070,3.506557,4.498368
22683,FOODS_3_206_CA_2,0.05,0.20,1.249377,124.1617,NaN,85.3714,3.569649,4.438610
22684,FOODS_3_206_CA_2,0.05,0.25,1.253514,124.5729,NaN,82.0833,3.581470,4.378942
...,...,...,...,...,...,...,...,...,...
28183,FOODS_3_444_CA_2,0.90,0.70,0.875000,100.0000,NaN,100.0000,1.000000,2.828427
28184,FOODS_3_444_CA_2,0.90,0.75,0.875000,100.0000,NaN,100.0000,1.000000,2.828427
28185,FOODS_3_444_CA_2,0.90,0.80,0.875000,100.0000,NaN,100.0000,1.000000,2.828427
28186,FOODS_3_444_CA_2,0.90,0.85,0.875000,100.0000,NaN,100.0000,1.000000,2.828427


In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [25]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [26]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [27]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [28]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.0 %
Percentage of unexpected values of WMAPE is: 0.0 %
Percentage of unexpected values of SMAPE is: 3.3057851239669422 %
Percentage of unexpected values of MAPE is: 0.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [29]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

# Get MEAN metrics value of each Alpha and Beta

In [30]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE       WMAPE      SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                                
0.05  0.05  1.326841  115.139345  89.866882  73.530463  4.234784  5.091263
      0.10  1.270296  109.330362  88.124838  72.935163  4.089388  4.973370
      0.15  1.226820  104.895085  87.512730  72.167739  4.011216  4.912237
      0.20  1.190283  101.138884  87.147548  71.295708  3.958637  4.871923
      0.25  1.162718   98.390205  87.021849  70.898252  3.921526  4.843941
...              ...         ...        ...        ...       ...       ...
0.90  0.70  0.995496   79.917920  94.035893  73.376446  3.838622  4.820611
      0.75  0.994885   79.982196  94.631611  73.939803  3.841312  4.830370
      0.80  0.994800   80.116392  95.278173  74.533959  3.845026  4.841225
      0.85  0.995584   80.370991  96.054255  75.262997  3.850011  4.853182
      0.90  0.997308   80.737052  97.062245  76.172973  3.856440  4.866281

[324 rows x 6 columns]

In [31]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.45, 0.8)
The optimum Alpha and Beta based on Mean WMAPE is: (0.5, 0.7)
The optimum Alpha and Beta based on Mean SMAPE is: (0.25, 0.25)
The optimum Alpha and Beta based on Mean MAPE is: (0.45, 0.05)
The optimum Alpha and Beta based on Mean MAE is: (0.35, 0.65)
The optimum Alpha and Beta based on Mean RMSE is: (0.35, 0.5)


# Get MEDIAN metrics value of each Alpha and Beta

In [32]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE    WMAPE    SMAPE     MAPE       MAE      RMSE
Alpha Beta                                                         
0.05  0.05  1.040157  67.2447  80.3790  52.1074  2.985036  3.574341
      0.10  1.015660  65.0416  71.9766  51.8297  2.911544  3.537920
      0.15  0.984952  63.1467  73.3612  54.0128  2.927208  3.459282
      0.20  1.010442  66.5303  72.3474  52.1484  2.920934  3.423313
      0.25  0.979662  66.0886  72.4929  51.1431  2.818269  3.382457
...              ...      ...      ...      ...       ...       ...
0.90  0.70  0.965091  67.4329  81.7145  65.3663  2.412423  3.211691
      0.75  0.964486  67.2254  83.6365  64.6614  2.392655  3.160068
      0.80  0.964486  65.0349  85.4979  65.3663  2.428200  3.141764
      0.85  0.961726  66.3437  86.0604  66.5114  2.430392  3.144807
      0.90  0.959116  66.8576  86.4766  68.3119  2.435511  3.211691

[324 rows x 6 columns]

In [33]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.35, 0.5)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.85)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.55)
The optimum Alpha and Beta based on Median of MAPE is: (0.4, 0.05)
The optimum Alpha and Beta based on Median of MAE is: (0.8, 0.65)
The optimum Alpha and Beta based on Median of RMSE is: (0.25, 0.75)


# Run Croston TSB model after decide best paramaters

In [13]:
best_alpha = 0.45
best_beta = 0.05

In [14]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.020807136666666656 minutes


In [15]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOBBIES_1_011_CA_1,1.0,0.456103
1,HOBBIES_1_011_CA_1,2.0,0.458885
2,HOBBIES_1_018_CA_1,0.0,0.134748
3,HOBBIES_1_018_CA_1,0.0,0.128010
4,HOBBIES_1_026_CA_1,0.0,0.000000
...,...,...,...
1733,HOBBIES_2_144_CA_4,0.0,0.148389
1734,HOBBIES_2_145_CA_4,0.0,0.800778
1735,HOBBIES_2_145_CA_4,0.0,0.760740
1736,HOBBIES_2_146_CA_4,0.0,0.437232


In [16]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [17]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_3,inf,36.4430,30.2294,36.4430,0.364430,0.390456
1,HOBBIES_1_002_CA_4,0.555271,111.0541,124.0731,38.8048,1.110541,1.159808
2,HOBBIES_1_003_CA_2,NaN,NaN,NaN,NaN,0.000000,0.000000
3,HOBBIES_1_003_CA_3,NaN,NaN,NaN,NaN,0.000000,0.000000
4,HOBBIES_1_003_CA_4,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...
864,HOBBIES_2_146_CA_1,0.519960,103.9919,136.7543,53.7524,1.039919,1.040512
865,HOBBIES_2_146_CA_2,0.525046,87.5076,109.2588,92.7023,5.250455,6.761968
866,HOBBIES_2_146_CA_3,0.585250,16.7214,17.0586,15.2694,0.585250,0.727477
867,HOBBIES_2_146_CA_4,inf,inf,200.0000,NaN,0.426301,0.426442


In [18]:
df_result_final.isnull().sum()

Product      0
MASE       132
WMAPE      132
SMAPE      132
MAPE       393
MAE          0
RMSE         0
dtype: int64

In [19]:
df_result_final.MAPE.mean()

57.336116596638654

In [20]:
df_result_final.to_csv('Croston_TSB_Intermittent_Metrics.csv')